aqui especificar el ETL y hago los requerimientos que me piden en el readme del github... acomodar introduccion

In [83]:
# Importar las bibliotecas necesarias
import numpy as np
import pandas as pd
import json
import ast

In [117]:
#Extraer el dataset
df_movies = pd.read_csv('Dataset_original/movies_dataset.csv', encoding='utf-8')

C:\Users\evely\AppData\Local\Temp\ipykernel_13176\1250963378.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('Dataset_original/movies_dataset.csv', encoding='utf-8')


In [86]:
#Imprimir tamaño y encabezados del dataset
print(df_movies.shape)
df_movies.head(1)

(45466, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [87]:

# Crear una copia del DataFrame original para no modificarlo directamente
df = df_movies.copy()


In [88]:
# Mostrar solo la columna 'belongs_to_collection'
print(df['belongs_to_collection'])

0        {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                      NaN
2        {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                      NaN
4        {'id': 96871, 'name': 'Father of the Bride Col...
                               ...                        
45461                                                  NaN
45462                                                  NaN
45463                                                  NaN
45464                                                  NaN
45465                                                  NaN
Name: belongs_to_collection, Length: 45466, dtype: object


In [89]:
# Función para desanidar el campo usando ast.literal_eval para la columna belongs_to_collection
def expand_collection(df, belongs_to_collection):
    def safe_literal_eval(x):
        if pd.isna(x) or x == '':
            return {}
        try:
            # Usar ast.literal_eval para convertir la cadena en diccionario
            data = ast.literal_eval(x)
            if isinstance(data, dict):
                return data
            return {}
        except (ValueError, SyntaxError):
            return {}
    
    # Aplicar la función para convertir los valores
    df[belongs_to_collection] = df[belongs_to_collection].apply(safe_literal_eval)
    
    # Crear nuevas columnas a partir del diccionario
    # solo se extrae informacion necesaria de esta columna
    df['idCollection'] = df[belongs_to_collection].apply(lambda x: x.get('id'))
    df['name_collection'] = df[belongs_to_collection].apply(lambda x: x.get('name'))
    
    return df.drop(columns=[belongs_to_collection])

# Aplicar la función
movies = expand_collection(df, 'belongs_to_collection')

# Mostrar las primeras filas para verificar
print(movies.head())


   adult    budget                                             genres  \
0  False  30000000  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  False  65000000  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  False         0  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  False  16000000  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4  False         0                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497                en   
2                                   NaN  15602  tt0113228                en   
3                                   NaN  31357  tt0114885                en   
4                                   NaN  11862  tt0113041                en   

                original_title  \
0                    Toy Story   
1                 

In [90]:
movies.head(1)

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,idCollection,name_collection
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,10194.0,Toy Story Collection


Ahora se procede a desanidar las demas columnas 

In [91]:
# Desanidar columnas del dataframe 

# Lista de columnas anidadas a desanidar
columnas_anidadas = ['genres', 'production_companies', 'production_countries', 'spoken_languages']

# Función para desanidar columnas que contienen listas de diccionarios
def desanidar_columna(df, columna):
    # Convertir cadenas de texto en listas/diccionarios si es necesario
    df[columna] = df[columna].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    
    # Desanidar según el tipo de datos en la columna
    if df[columna].apply(lambda x: isinstance(x, list)).any():
        # Para columnas con listas de diccionarios
        desanidado = pd.json_normalize(df[columna].explode().dropna()).reset_index(drop=True)
    else:
        # Para columnas con diccionarios simples
        desanidado = pd.json_normalize(df[columna].dropna()).reset_index(drop=True)
    
    # Renombrar las columnas desanidadas
    desanidado.columns = [f"{columna}_{subcol}" for subcol in desanidado.columns]
    
    # Unir las columnas desanidadas al DataFrame original
    df = df.drop(columns=[columna]).join(desanidado)
    return df

# Desanidar columnas que contienen listas de diccionarios
for columna in ['genres', 'production_companies', 'production_countries', 'spoken_languages']:
    movies = desanidar_columna(movies, columna)


In [92]:
print(movies.shape)

movies.head(1)


(45466, 29)


,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,idCollection,name_collection,genres_id,genres_name,production_companies_name,production_companies_id,production_countries_iso_3166_1,production_countries_name,spoken_languages_iso_639_1,spoken_languages_name
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,10194.0,Toy Story Collection,16,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English


In [93]:
#Se deben eliminar columnas de las desanidadas que no se usaran ***

In [94]:
# Eliminar las columnas que no serán utilizadas
columns_to_drop = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage',  
                   'production_countries_iso_3166_1', 'spoken_languages_iso_639_1']
movies = movies.drop(columns=columns_to_drop)

In [95]:

# Renombrar algunas columnas
movies = movies.rename(columns={
    'id': 'idMovies',
    'genres_id': 'id_genres',
    'genres_name': 'genres',
    'production_companies_name': 'production_companies',
    'production_companies_id': 'id_production',
    'production_countries_name': 'production_countries',
    'spoken_languages_name': 'spoken_language'

})

# Verificar las primeras filas del DataFrame para asegurarse de que las columnas han sido renombradas
movies.head(1)

,budget,idMovies,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,vote_average,vote_count,idCollection,name_collection,id_genres,genres,production_companies,id_production,production_countries,spoken_language
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,7.7,5415.0,10194.0,Toy Story Collection,16,Animation,Pixar Animation Studios,3.0,United States of America,English


In [96]:
#
print(movies.shape)
movies.head(1)

(45466, 21)


,budget,idMovies,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,vote_average,vote_count,idCollection,name_collection,id_genres,genres,production_companies,id_production,production_countries,spoken_language
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,7.7,5415.0,10194.0,Toy Story Collection,16,Animation,Pixar Animation Studios,3.0,United States of America,English


Transformaciones solicitadas

In [97]:
# Rellenar los valores nulos en las columnas 'revenue' y 'budget' con 0

movies['revenue'] = movies['revenue'].fillna(0)
movies['budget'] = movies['budget'].fillna(0)

# Ver los primeros registros para verificar
print(movies[['revenue', 'budget']].head())


       revenue    budget
0  373554033.0  30000000
1  262797249.0  65000000
2          0.0         0
3   81452156.0  16000000
4   76578911.0         0


In [98]:
# Eliminar las filas donde el campo 'release date' es nulo
movies = movies.dropna(subset=['release_date'])

# Verificar los primeros registros para asegurarse de que se han eliminado las filas nulas
print(movies[['release_date']].head())

  release_date
0   1995-10-30
1   1995-12-15
2   1995-12-22
3   1995-12-22
4   1995-02-10


In [99]:
# Convertir la columna 'release_date' al formato de fecha AAAA-mm-dd
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce', format='%Y-%m-%d')

# Crear la columna 'release_year' extrayendo el año de 'release_date' y convirtiendo a formato de fecha con frecuencia anual
movies['release_year'] = movies['release_date'].dt.to_period('Y')

# Opcional: Eliminar filas con 'release_year' nulo
movies = movies.dropna(subset=['release_year'])

# Verificar los primeros registros para asegurarse de que las fechas están formateadas y el año ha sido extraído
print(movies[['release_date', 'release_year']].head())


  release_date release_year
0   1995-10-30         1995
1   1995-12-15         1995
2   1995-12-22         1995
3   1995-12-22         1995
4   1995-02-10         1995


In [100]:
#Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, 
# dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0

# Asegurarse de que las columnas 'revenue' y 'budget' sean numéricas
movies['revenue'] = pd.to_numeric(movies['revenue'], errors='coerce')
movies['budget'] = pd.to_numeric(movies['budget'], errors='coerce')

# Crear la columna 'return' calculando revenue / budget
# Si el budget es 0 o NaN, el retorno también debe ser 0
movies['return'] = movies.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] > 0 else 0, axis=1)

# Verificar los primeros registros para asegurarse de que el cálculo es correcto
print(movies[['revenue', 'budget', 'return']].head())


       revenue    budget     return
0  373554033.0  30000000  12.451801
1  262797249.0  65000000   4.043035
2          0.0         0   0.000000
3   81452156.0  16000000   5.090760
4   76578911.0         0   0.000000


Realizar otras modificaciones necesarias

In [101]:
# Verifica los primeros registros para asegurarte de que la desanidación se realizó correctamente****
print(movies[['idCollection','name_collection']].head())

   idCollection                 name_collection
0       10194.0            Toy Story Collection
1           NaN                            None
2      119050.0       Grumpy Old Men Collection
3           NaN                            None
4       96871.0  Father of the Bride Collection


In [102]:
#Verificacion de los tipos de datos
print(movies.dtypes)

budget                           int64
idMovies                        object
original_language               object
overview                        object
popularity                      object
release_date            datetime64[ns]
revenue                        float64
runtime                        float64
status                          object
tagline                         object
title                           object
vote_average                   float64
vote_count                     float64
idCollection                   float64
name_collection                 object
id_genres                        int64
genres                          object
production_companies            object
id_production                  float64
production_countries            object
spoken_language                 object
release_year             period[Y-DEC]
return                         float64
dtype: object


In [103]:
# Modificación de idMovies
# Convertir la columna 'id' a entero (int)
movies['idMovies'] = movies['idMovies'].astype(int)


# Modificación de idProduccion
# Llenar los valores nulos en 'idProduccion' con 0
movies['id_production'] = movies['id_production'].fillna(0)

# Convertir la columna 'idProduccion' a entero
movies['id_production'] = movies['id_production'].astype(int)


In [104]:
# Convertir 'popularity' a float64
movies['popularity'] = movies['popularity'].astype(float)

# Verificar los tipos de datos después de las conversiones
print(movies.dtypes)

budget                           int64
idMovies                         int32
original_language               object
overview                        object
popularity                     float64
release_date            datetime64[ns]
revenue                        float64
runtime                        float64
status                          object
tagline                         object
title                           object
vote_average                   float64
vote_count                     float64
idCollection                   float64
name_collection                 object
id_genres                        int64
genres                          object
production_companies            object
id_production                    int32
production_countries            object
spoken_language                 object
release_year             period[Y-DEC]
return                         float64
dtype: object


Verificacion de Campos Nulos

In [105]:
#valores nulos
movies.isnull().sum()

budget                      0
idMovies                    0
original_language          11
overview                  941
popularity                  0
release_date                0
revenue                     0
runtime                   246
status                     80
tagline                 24978
title                       0
vote_average                0
vote_count                  0
idCollection            40888
name_collection         40888
id_genres                   0
genres                      0
production_companies        1
id_production               0
production_countries        3
spoken_language             0
release_year                0
return                      0
dtype: int64

In [106]:
print(movies['name_collection'].unique())

['Toy Story Collection' None 'Grumpy Old Men Collection' ...
 'Ducobu Collection' 'Mister Blot Collection' 'Red Lotus Collection']


Resumen: Eliminación de la Columna coleccion_de_peliculas

La columna coleccion_de_peliculas fue eliminada del conjunto de datos debido a su bajo nivel de cobertura y relevancia para el proyecto. Debido a su alta proporción de Valores Nulos: De las 45,466 películas en el conjunto de datos, 40,888 (aproximadamente el 90%).Esto indica que la mayoría de las películas no pertenecen a una franquicia o colección identificada, lo que limita significativamente su utilidad como característica predictiva para el sistema de recomendación.


In [107]:
# Eliminar las columnas que no serán utilizadas
columns_to_drop = ['idCollection', 'name_collection']
movies = movies.drop(columns=columns_to_drop)

In [108]:
#valores nulos
movies.isnull().sum()

budget                      0
idMovies                    0
original_language          11
overview                  941
popularity                  0
release_date                0
revenue                     0
runtime                   246
status                     80
tagline                 24978
title                       0
vote_average                0
vote_count                  0
id_genres                   0
genres                      0
production_companies        1
id_production               0
production_countries        3
spoken_language             0
release_year                0
return                      0
dtype: int64

In [109]:
# Diccionario con columnas y valores de relleno
fill_values = {
    
    'original_language': 'unknown',
    'overview': 'no summary',
    'tagline': 'no summary',
    'status': 'unknown',
    'production_companies': 'unknown',
    'production_countries': 'unknown',
    'runtime': 0
    
}

# Rellenar los valores nulos en las columnas especificadas usando un diccionario
movies.fillna(value=fill_values, inplace=True)

In [110]:
movies.shape


(45376, 21)

In [111]:
#valores nulos
movies.isnull().sum()

budget                  0
idMovies                0
original_language       0
overview                0
popularity              0
release_date            0
revenue                 0
runtime                 0
status                  0
tagline                 0
title                   0
vote_average            0
vote_count              0
id_genres               0
genres                  0
production_companies    0
id_production           0
production_countries    0
spoken_language         0
release_year            0
return                  0
dtype: int64

Verificar duplicados


In [112]:

# Verificar duplicados basados en la columna 'idMovies' 
duplicates_by_id = movies[movies.duplicated(subset=['idMovies'], keep=False)]

# Verificar duplicados en todas las columnas
duplicates_all = movies[movies.duplicated(keep=False)]

# Mostrar duplicados
print("Duplicados basados en la columna 'idMovies' de la tabla Movies:", len(duplicates_by_id))
print("Duplicados en todas las columnas:", len(duplicates_all))


Duplicados basados en la columna 'idMovies' de la tabla Movies: 59
Duplicados en todas las columnas: 0


In [113]:
duplicates = movies[movies.duplicated(subset=['idMovies'], keep=False)]
print(duplicates)

         budget  idMovies original_language  \
676           0    105045                de   
838           0    132641                ja   
949           4     22649                en   
1465          0    105045                de   
2564          0     84198                en   
4114   16000000     10991                ja   
4356    3512454    110428                fr   
5130   26000000     15028                en   
5535          0     12600                ja   
5710          0    109962                en   
5865   30000000      4912                en   
7345          0      5511                fr   
8068          0     23305                en   
9165          0      5511                fr   
9327          0     23305                en   
9576   10000000     69234                en   
10419   1600000     14788                en   
11155  40000000     77221                en   
11342      2500     13209                fa   
12066   1600000     14788                en   
13220        

Duplicados: Tienes 29 películas con duplicados, resultando en un total de 59 registros duplicados.
Presupuesto: Hay una gran variabilidad en el presupuesto entre los duplicados, con muchos registros con presupuestos bajos o nulos y algunos con presupuestos muy altos.
Idiomas: La mayoría de los registros duplicados están en inglés, con algunos en otros idiomas.
Este resumen te ayuda a comprender el impacto y la naturaleza de los registros duplicados en tu dataset y puede guiarte en cómo manejar estos duplicados en tu análisis.


Se recomienda hacerlo sobre la columna idMovies, que parece ser un identificador único para las películas.

In [114]:
# Eliminar duplicados basados en la columna 'idMovies'
movies2 = movies.drop_duplicates(subset='idMovies', keep='first')

# Confirmar la eliminación de duplicados
print(f"Número de filas después de eliminar duplicados: {movies2.shape[0]}")


Número de filas después de eliminar duplicados: 45346


In [115]:
# Verificar duplicados basados en la columna 'idMovies' 
duplicates_by_id = movies2[movies2.duplicated(subset=['idMovies'], keep=False)]

# Mostrar duplicados
print("Duplicados basados en la columna 'idMovies' de la tabla Movies:", len(duplicates_by_id))


Duplicados basados en la columna 'idMovies' de la tabla Movies: 0


In [116]:

# Exportar a archivo Parquet usando pyarrow
movies2.to_parquet('movies_modificado.parquet', engine='pyarrow')

pyarrow es una biblioteca de Python que proporciona herramientas para trabajar con datos en formato Apache Arrow y Parquet.
Apache Arrow es un formato en memoria que permite el procesamiento de datos de manera eficiente, mientras que Parquet es un formato de archivo columnar que es ideal para grandes conjuntos de datos.

ETL DE DATASETS DE CREDITS

In [118]:
#Extraer el dataset
df_cast = pd.read_csv('Dataset_original/cast_credits.csv', encoding='utf-8')
df_crew = pd.read_csv('Dataset_original/crew_credits.csv', encoding='utf-8')

1. Primero se trabajara con el dataset de cast que contiene informacion detallada de los actores

In [158]:
#Imprimir tamaño y encabezados del dataset
print(df_cast.shape)
df_cast.head(1)

(562474, 9)


,cast_cast_id,cast_character,cast_credit_id,cast_gender,cast_id,cast_name,cast_order,cast_profile_path,id
0,14,Woody (voice),52fe4284c3a36847f8024f95,2,31,Tom Hanks,0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,862.0


In [157]:
# Crear una copia del DataFrame original para no modificarlo directamente
cast = df_cast.copy()

In [159]:
# Eliminar las columnas que no serán utilizadas
columns_to_drop = ['cast_credit_id', 'cast_gender', 'cast_profile_path']
cast_df = cast.drop(columns=columns_to_drop)

In [160]:
# Renombrar algunas columnas
cast = cast_df.rename(columns={
    'cast_cast_id': 'cast_id',
    'cast_character	': 'character',
    'cast_id': 'IdActor',
    'cast_name': 'name_actor',
    
})

# Verificar las primeras filas del DataFrame para asegurarse de que las columnas han sido renombradas
cast.head(1)

,cast_id,cast_character,IdActor,name_actor,cast_order,id
0,14,Woody (voice),31,Tom Hanks,0,862.0


** Verificacion de los tipos de datos

In [161]:
print(cast.dtypes)

cast_id             int64
cast_character     object
IdActor             int64
name_actor         object
cast_order          int64
id                float64
dtype: object


In [162]:
# Modificación de id

# Renombrar la columna de id
cast.rename(columns={'id': 'idMovies'}, inplace=True)



In [163]:
# Llenar los valores nulos en 'idProduccion' con 0
cast['idMovies'] = cast['idMovies'].fillna(0)

# Convertir la columna 'idProduccion' a entero
cast['idMovies'] = cast['idMovies'].astype(int)

In [164]:
print(cast.dtypes)

cast_id            int64
cast_character    object
IdActor            int64
name_actor        object
cast_order         int64
idMovies           int32
dtype: object


** Verificacion de campos Nulos

In [165]:
#valores nulos
cast.isnull().sum()

cast_id               0
cast_character    25783
IdActor               0
name_actor            0
cast_order            0
idMovies              0
dtype: int64

In [166]:
cast['cast_character'] = cast['cast_character'].fillna('unknown')


In [167]:
#valores nulos
cast.isnull().sum()

cast_id           0
cast_character    0
IdActor           0
name_actor        0
cast_order        0
idMovies          0
dtype: int64

** Verificar los campos duplicados

In [168]:
# Verificar duplicados en 'cast_credits'
cast_duplicados = cast.duplicated(subset='idMovies', keep=False)
print(f"Número de duplicados en cast_credits: {cast_duplicados.sum()}")

Número de duplicados en cast_credits: 517085


La acción en este casos es eliminar los duplicados directos, ya que tener múltiples entradas idénticas para una película y un actor puede sesgar tu sistema de recomendación, haciendo que ciertas películas o actores parezcan más relevantes de lo que realmente son.

In [169]:
# Eliminar duplicados basados en 'idMovies'
actors_df = cast.drop_duplicates(subset=['idMovies'])


In [170]:

# Filtrar el dataset de actores para mantener solo las películas que están en el dataset de movies
filtered_actors_df = actors_df[actors_df['idMovies'].isin(movies['idMovies'])]

# Verificar el tamaño del dataset filtrado
print(filtered_actors_df.shape)

(45345, 6)


In [171]:
# Verificar tamaños
print("Tamaño del dataset de películas:", movies.shape)
print("Tamaño después de filtrar actores:", filtered_actors_df.shape)

Tamaño del dataset de películas: (45376, 21)
Tamaño después de filtrar actores: (45345, 6)


In [189]:
# Verificar diferencias
missing_movies_ids = set(movies['idMovies']) - set(actors_df['idMovies'])
print(f"Cantidad de idMovies en movies.csv pero no en filtered_actors_df: {len(missing_movies_ids)}")


Cantidad de idMovies en movies.csv pero no en filtered_actors_df: 1


In [190]:

# Exportar a archivo Parquet usando pyarrow
actors_df.to_parquet('credits_cast.parquet', engine='pyarrow')

2. ETL DE DATASET DE CREW_CREDITS

** Primero se trabajara con el dataset de cast que contiene informacion detallada de los actores

In [155]:
#Imprimir tamaño y encabezados del dataset
print(df_crew.shape)
df_crew.head(1)

(464314, 8)


,crew_credit_id,crew_department,crew_gender,crew_id,crew_job,crew_name,crew_profile_path,id
0,52fe4284c3a36847f8024f49,Directing,2,7879,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg,862.0


In [156]:
# Crear una copia del DataFrame original para no modificarlo directamente
crew = df_crew.copy()

In [174]:
# Eliminar las columnas que no serán utilizadas
columns_to_drop = ['crew_credit_id', 'crew_gender', 'crew_profile_path']
crew_df = crew.drop(columns=columns_to_drop)

In [176]:
crew_df.head(1)

,crew_department,crew_id,crew_job,crew_name,id
0,Directing,7879,Director,John Lasseter,862.0


** Verificacion de los tipos de datos

In [177]:
print(crew_df.dtypes)

crew_department     object
crew_id              int64
crew_job            object
crew_name           object
id                 float64
dtype: object


In [178]:

# Modificación de id

# Renombrar la columna 'A' a 'Columna1'
crew_df.rename(columns={'id': 'idMovies'}, inplace=True)



In [180]:
# Llenar los valores nulos en 'idMovies' con 0
crew_df['idMovies'] = crew_df['idMovies'].fillna(0)

# Convertir la columna 'idProduccion' a entero
crew_df['idMovies'] = crew_df['idMovies'].astype(int)

In [181]:
print(crew_df.dtypes)

crew_department    object
crew_id             int64
crew_job           object
crew_name          object
idMovies            int32
dtype: object


** Verificacion de campos Nulos

In [182]:
#valores nulos
crew_df.isnull().sum()

crew_department    0
crew_id            0
crew_job           0
crew_name          0
idMovies           0
dtype: int64

** Verificar los campos duplicados

In [183]:
# Verificar duplicados en 'crew_credits'
crew_duplicados = crew_df.duplicated(subset='idMovies', keep=False)
print(f"Número de duplicados en cast_credits: {crew_duplicados.sum()}")

Número de duplicados en cast_credits: 418925


In [184]:
# Eliminar duplicados basados en 'idMovies'
crew = crew_df.drop_duplicates(subset=['idMovies'])

In [185]:

# Filtrar el dataset de reparto para mantener solo las películas que están en el dataset de movies
filtered_crew = crew[crew['idMovies'].isin(movies['idMovies'])]

# Verificar el tamaño del dataset filtrado
print(filtered_crew.shape)

(45345, 5)


In [186]:
# Verificar tamaños
print("Tamaño del dataset de películas:", movies.shape)
print("Tamaño después de filtrar actores:", filtered_crew.shape)

Tamaño del dataset de películas: (45376, 21)
Tamaño después de filtrar actores: (45345, 5)


In [191]:
# Verificar diferencias
missing_movies_ids = set(movies['idMovies']) - set(crew['idMovies'])
print(f"Cantidad de idMovies en movies.csv pero no en filtered_actors_df: {len(missing_movies_ids)}")

Cantidad de idMovies en movies.csv pero no en filtered_actors_df: 1


In [192]:

# Exportar a archivo Parquet usando pyarrow
crew.to_parquet('credits_crew.parquet', engine='pyarrow')